# julia itteration solver sparse src

In [ ]:
#copyright kkk

In [1]:
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate
using SparseArrays, Arpack, LinearAlgebra
using BenchmarkTools, IterativeSolvers, MatrixDepot, Random


struct DiagonalIndices{Tv, Ti <: Integer}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::Vector{Ti}

    function DiagonalIndices{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti}
        # Check square?
        diag = Vector{Ti}(undef, A.n)

        for col = 1 : A.n
            r1 = Int(A.colptr[col])
            r2 = Int(A.colptr[col + 1] - 1)
            r1 = searchsortedfirst(A.rowval, col, r1, r2, Base.Order.Forward)
            if r1 > r2 || A.rowval[r1] != col || iszero(A.nzval[r1])
                throw(LinearAlgebra.SingularException(col))
            end
            diag[col] = r1
        end 

        new(A, diag) #
    end
end

DiagonalIndices(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti} = DiagonalIndices{Tv,Ti}(A)
@inline getindex(d::DiagonalIndices, i::Int) = d.diag[i]


struct FastLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct FastUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct OffDiagonal{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end


function forward_sub!(F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix
    @inbounds for col = 1 : A.n
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # ok
        for i = idx + 1 : (A.colptr[col + 1] - 1) #colptr인데 lower triangular이기 때문에 해당 col의 diagonal 아래 개수가나옴.
            x[A.rowval[i]] -= A.nzval[i] * x[col] # 이 term으로 x[n] 계산할때 그이전텀들이 다 마이너스 되어서 있음. 
        end
    end
    x
end

function backward_sub!(F::FastUpperTriangular, x::AbstractVector)
    A = F.matrix

    @inbounds for col = A.n : -1 : 1

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] = x[col] / A.nzval[idx]

        # Substitute next values involving x[col]
        for i = A.colptr[col] : idx - 1
            x[A.rowval[i]] -= A.nzval[i] * x[col]
        end
    end

    x
end




┌ Info: Recompiling stale cache file C:\Users\cobook\.julia\compiled\v1.2\BenchmarkTools\ZXPQo.ji for BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\cobook\.julia\compiled\v1.2\MatrixDepot\90RGs.ji for MatrixDepot [b51810bb-c9f3-55da-ae3c-350fc1fbce05]
└ @ Base loading.jl:1240


include group.jl for user defined matrix generators
verify download of index files...
used remote site is https://sparse.tamu.edu/?per_page=All
populating internal database...


backward_sub! (generic function with 1 method)

## Matrix Generator

In [2]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random


using LinearAlgebra
using SparseArrays

function cartesianidx(k::Int, n::Int)
    """
    {s_i} are on a regular 10 by 10 lattice over the two dimensional domain
    S = [1, 10] \times [1, 10].
    output gives a cartesian index (i, j) of s_k, i.e., 
    S = [s01 s11 s21 ... s91
         s02 s12 s22 ... s92
         s03 s13 s23 ... s93
            ... ... 
         s10 s20 s30 ... s100]
    """
    if k % n == 0
        [n; k ÷ n]
    else
        [k % n; k ÷ n + 1]
    end
end

"""
n_i is the number of points neighbouring s_i, i.e., with distance 1from s_i
N[i, j] = n_k, where cartesianindex(k, 10) = (i, j)
"""

function gen_Ab(n)
    #n = 10
    N = 4 * ones(n, n);
    for k in 1:n
        N[1, k] -= 1
        N[k, 1] -= 1
        N[n, k] -= 1
        N[k, n] -= 1
    end

    """
    6.1 A 10 by 10 lattice example (n = 100)
    """
    A = zeros(n^2, n^2)
    for i in 1:n^2
        for j in 1:n^2
            if i == j
                A[i, j] = 0.0001 + N[i]
            elseif norm(cartesianidx(i, n)-cartesianidx(j, n)) <= 1.0
                A[i, j] = -1.0
            else
                A[i, j] = 0.0
            end
        end
    end

    A = sparse(A)
    b = randn(n^2)
    
    (A,b)
end



function gen_Ab_temp(n)
    #n = 100
    # Poisson matrix of dimension n^2=10000, pd and sparse
    A = matrixdepot("poisson", n)
    @show typeof(A)
    # dense matrix representation of A
    #Afull = convert(Matrix, A)
    #@show typeof(Afull)
    # sparsity level
    count(!iszero, A) / length(A)
    b = randn(n^2)
    (A,b)
end

gen_Ab_temp (generic function with 1 method)

# K3 src

# SOR

In [3]:
function f_mul!(α::T, O::DiagonalIndices, x::AbstractVector, b::AbstractVector, r::AbstractVector  ) where {T}
    A = O.matrix
    r[:] = b
    @inbounds for col = 1 : A.n
        αx = α * x[col]
        diag_index = O.diag[col]
        for j = A.colptr[col] : A.colptr[col + 1] - 1
            r[A.rowval[j]] += A.nzval[j] * αx 
        end
    end
    r
end

function sum!(z, r, A::SparseMatrixCSC)
    @inbounds for i =1: A.n
        z[i] += r[i]
    end
end


function m_sor!(A, D::DiagonalIndices, w)
    for d_idx in D.diag 
        A.nzval[d_idx]  *= (1/w)
    end
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_sor!(F::FastLowerTriangular, D::DiagonalIndices,
                        x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("sor_itter : ",i)
            break
        end 
        
        forward_sub!(F, r)# r <- M_sor\r
        sum!(x, r, A) # x <- x +  M_sor/b        
    end
    x
    
end

function k3_sor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    m_sor!(m_sor, D, w)
    D_ms = DiagonalIndices(m_sor)
    itter_sor!(FastLowerTriangular(m_sor ,D_ms), D, x , b, maxiter)
end



k3_sor (generic function with 1 method)

# SSOR

In [12]:
function gamma_sqrt_diag_mul!( D::DiagonalIndices, b::AbstractVector, w ,b_c)
    A = D.matrix
    for idx in D.diag 
        b[A.rowval[idx]] *=  sqrt( b_c * ((2/w) -1) * A.nzval[idx])
    end
end

function sum2!(x,y,z, A::SparseMatrixCSC)
    @inbounds for i =1: A.n
        x[i] =y[i]+z[i]
    end
end

function itter_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  max_itter)
    
    A = D.matrix
    A_t = D_t.matrix
    #symetric일때도 필요한지 고려 diag정의는 새로필요한거 같음
    
    T = eltype(b)
    r = zeros(A.n)
    y = zeros(A.n)
        
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r_1 <-  γ * D^(1/2) * b- Ay

        """
        if norm(r) < 10^(-8)
            println("ssor_itter : ",i)
            break
        end 
        
        """
        
        gamma_sqrt_diag_mul!(D,r,w,1)
        forward_sub!(F, r) #r_1 <- m_sor\r_1
        gamma_sqrt_diag_mul!(D,r,w,1)
        backward_sub!(U, r)
        sum!(x, r, A)

    end
    x
end



function k3_ssor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    
    m_sor!(m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    
    itter_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, maxiter)
end


k3_ssor (generic function with 1 method)

# Cheby_SSOR

In [5]:
mutable struct CB_variable
    β::Float64
    α::Float64
    b::Float64
    a::Float64
    κ::Float64
end

function mul_sum!(x_next, x, x_pre, r, α, τ, A)
    for i = 1:A.n 
        x_next[i] = (1-α)*x_pre[i] + α*x[i] + (τ*α*r[i])
    end
    # x
end



function sum2!(x_next, x, r, A)
    for i = 1:A.n 
        x_next[i] = x[i] + r[i]
    end
    # x
end

function sum3!(w_v, x_temp , x, r, A)
    for i = 1:A.n 
        w_v[i] = x_temp[i] - x[i] + r[i]
    end
    # x
end

function eigMm(A::SparseMatrixCSC, ω::Real)

    Dw = sqrt((2/ω-1)) * Diagonal(sqrt.(diag(A)))
    L = (LowerTriangular(A)- (1-1/ω) * Diagonal(A))*(inv(Dw))

    Meig = inv(cholesky(L*L')) * A
    
    λ_max = eigs(Meig; nev=1, ritzvec=false, which=:LM)[1][1]
    λ_min = eigs(Meig; nev=1, ritzvec=false, which=:SM)[1][1]
    real(λ_max), real(λ_min)
end


function itter_CB_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  λ_max, λ_min, max_itter)
    
    A = D.matrix
    A_t = D_t.matrix
    
    δ = ((λ_max - λ_min)/4)^2
    τ = 2/(λ_max + λ_min)
    
    T = eltype(b)
    cb = CB_variable(0,0,0,0,0)
    #Assign initial parameter
    cb.β  = 2*τ
    cb.α = 1
    cb.b = 2/cb.α - 1
    cb.a = (2/τ -1) * cb.b
    cb.κ = τ
    
    T = eltype(b)
    r_1 = zeros(A.n)
    r_2 = zeros(A.n)
    x_pre = zeros(A.n)
    x_next = zeros(A.n)
    x_temp = zeros(A.n)
    w_v = zeros(A.n)
 
    for i = 1 : max_itter 

        x_pre[:] = x 
        x[:] = x_next
        
        f_mul!(-one(T), D, x, b, r_1) # r <- b - A* X        
        if norm(r_1) < 10^(-8)
            println("CB sor_itter : ",i)
            break
        end 
        
        
        
        
        forward_sub!(F, r_1) #r_1 <- m_sor\r_1
        sum2!(x_temp, x, r_1, A) #x_next <- x + τ*r
        
        f_mul!(-one(T), D, x_temp, b, r_2) # r_2 <- b - A* X
        backward_sub!(U, r_2)
        sum3!(w_v, x_temp , x, r_2, A)
        
        if i == 1
            sum2!(x_next, cb.α*x, τ*w_v, A) #x_next <- x + τ*r
        else
            mul_sum!(x_next, x, x_pre, w_v, cb.α, τ, A) # x_next <- (1-α)*x_pre + α*x + (τ*α*r[i])

        end
        
        
        cb.β = 1 / ( (1/τ)  - cb.β*δ ) 
        cb.α = cb.β / τ
        cb.b = ( ( 2*cb.κ*(1- cb.α) ) / cb.β) + 1
        cb.a = ((2/τ) -1) + (cb.b-1) * ( (1/τ) + (1/cb.κ) -1 )
        cb.κ = cb.β  + ( (1 - cb.α) * cb.κ)
        
    end
    x

end



function k3_CB_ssor(A, b::AbstractVector, w, λ_max, λ_min,  maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    
    m_sor!(m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    #λ_max,λ_min = eigMm(A, w)
    
    itter_CB_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, λ_max, λ_min, maxiter)
end


k3_CB_ssor (generic function with 1 method)

# Richardson

In [6]:

function itter_Richardson!(D::DiagonalIndices, x::AbstractVector, w, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("Richardson_itter : ",i)
            break
        end 
        
        sum!(x, w*r, A) # x <- x +  wr
        
    end
    x
    
end


function k3_RCS(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    D = DiagonalIndices(A)
    itter_Richardson!(D, x , w, b, maxiter)
end



k3_RCS (generic function with 1 method)

# Jacobi

In [7]:
function mul_inv_d!(D, r)
    A = D.matrix
    for idx in D.diag 
        r[A.rowval[idx]] *=  (1 / A.nzval[idx])
    end
end
    

function itter_JCB!(D::DiagonalIndices, x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("Jacobi_itter : ",i)
            break
        end 

        mul_inv_d!(D, r)# r <- D^-1*r
        sum!(x, r, A) # x <- x +  M^-1*r
    end
    x
    
end


function k3_JCB(A, b::AbstractVector, maxiter)
    x = zeros(A.n)
    D = DiagonalIndices(A)
    itter_JCB!(D, x , b, maxiter)
end



k3_JCB (generic function with 1 method)

# Gauss-Seidal

In [14]:

function m_GS!(A, D::DiagonalIndices)
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_GS!(F::FastLowerTriangular, D::DiagonalIndices,
                        x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(x)
    r =zeros(A.n)
    
    for i = 1 : max_itter 
        f_mul!(-one(T), D, x, b, r) # r <- b- Ax
        
        if norm(r) < 10^(-8)
            println("Gause-Seidal : ",i)
            break
        end 
        
        forward_sub!(F, r)# r <- M_sor\r
        sum!(x, r, A) # x <- x +  M_sor/b        
    end
    x
    
end


function k3_GS(A, b::AbstractVector, maxiter)
    x = zeros(A.n)
    m_GS = copy(A)
    D = DiagonalIndices(A)
    m_GS!(m_GS, D)
    D_GS = DiagonalIndices(m_GS)
    itter_GS!(FastLowerTriangular(m_GS ,D_GS), D, x , b, maxiter)
end



k3_GS (generic function with 1 method)

# Test

In [7]:
A,b = gen_Ab(10)

(
  [1  ,   1]  =  2.0001
  [2  ,   1]  =  -1.0
  [11 ,   1]  =  -1.0
  [1  ,   2]  =  -1.0
  [2  ,   2]  =  3.0001
  [3  ,   2]  =  -1.0
  [12 ,   2]  =  -1.0
  [2  ,   3]  =  -1.0
  [3  ,   3]  =  3.0001
  [4  ,   3]  =  -1.0
  [13 ,   3]  =  -1.0
  [3  ,   4]  =  -1.0
  ⋮
  [97 ,  97]  =  3.0001
  [98 ,  97]  =  -1.0
  [88 ,  98]  =  -1.0
  [97 ,  98]  =  -1.0
  [98 ,  98]  =  3.0001
  [99 ,  98]  =  -1.0
  [89 ,  99]  =  -1.0
  [98 ,  99]  =  -1.0
  [99 ,  99]  =  3.0001
  [100,  99]  =  -1.0
  [90 , 100]  =  -1.0
  [99 , 100]  =  -1.0
  [100, 100]  =  2.0001, [0.10966715721160032, 0.2656927127025517, 0.794014581258495, -0.0468183690652249, -0.8327446214722528, -1.468576823678516, 0.7977695961053077, -0.5941877059907252, -0.14393615532592963, -0.3783799871839709  …  -1.8064886300659826, -1.0278464284487296, -1.8251696419248158, 0.8072620598173083, 2.734476470340489, 0.7786217976088599, -1.6123564410967657, -0.3803267338952001, 1.2074555518725307, 0.025362889872946846])

In [15]:
@benchmark sor(A, b, 1.6641, maxiter= 64000)

BenchmarkTools.Trial: 
  memory estimate:  1.96 MiB
  allocs estimate:  128008
  --------------
  minimum time:     86.863 ms (0.00% GC)
  median time:      87.762 ms (0.00% GC)
  mean time:        89.777 ms (1.28% GC)
  maximum time:     133.410 ms (34.79% GC)
  --------------
  samples:          56
  evals/sample:     1

In [14]:
@benchmark k3_sor(A, b, 1.6641, 64000)

BenchmarkTools.Trial: 
  memory estimate:  12.02 KiB
  allocs estimate:  11
  --------------
  minimum time:     85.997 ms (0.00% GC)
  median time:      87.990 ms (0.00% GC)
  mean time:        89.472 ms (0.00% GC)
  maximum time:     100.938 ms (0.00% GC)
  --------------
  samples:          56
  evals/sample:     1

In [89]:
w = 1.6641
λ_max,λ_min = eigMm(A, w)
@benchmark k3_CB_ssor(A, b_, 1.6641, λ_max, λ_min, 1100)

BenchmarkTools.Trial: 
  memory estimate:  36.89 KiB
  allocs estimate:  33
  --------------
  minimum time:     3.112 ms (0.00% GC)
  median time:      3.273 ms (0.00% GC)
  mean time:        3.337 ms (1.31% GC)
  maximum time:     54.598 ms (93.92% GC)
  --------------
  samples:          1497
  evals/sample:     1

In [90]:
w = 1.6641
λ_max,λ_min = eigMm(A, w)
@benchmark k3_CB_ssor(A, b_, 1.6641, λ_max, λ_min, 80000)

BenchmarkTools.Trial: 
  memory estimate:  36.89 KiB
  allocs estimate:  33
  --------------
  minimum time:     236.739 ms (0.00% GC)
  median time:      240.880 ms (0.00% GC)
  mean time:        243.286 ms (0.00% GC)
  maximum time:     263.917 ms (0.00% GC)
  --------------
  samples:          21
  evals/sample:     1

In [15]:
sol_1 = sor(A,b , 1.9852 ,maxiter=100000)
sol_2 = ssor(A,b , 1.6641 ,maxiter=100000)


#b_ = copy(b)
#sol_3 = k3_RCS(A, b_, 1, 100000)


b_ = copy(b)
sol_4 = k3_JCB(A, b_, 1000000)


b_ = copy(b)
sol_5 = k3_ssor(A, b_, 1.6641, 100000)

b_ = copy(b)
sol_6 = k3_sor(A, b_, 1.9852, 100000)


w = 1
λ_max,λ_min = eigMm(A, w)
b_ = copy(b)
sol_7 = k3_CB_ssor(A, b_, 1, λ_max,λ_min, 100000)

w = 1.6641
λ_max,λ_min = eigMm(A, w)
b_ = copy(b)
sol_8 = k3_CB_ssor(A, b_, 1.6641, λ_max, λ_min, 80000)

b_ = copy(b)
sol_8 = k3_GS(A, b_, 1000000)

#copyright kkk

Jacobi_itter : 640842
ssor_itter : 55858
sor_itter : 1536
CB sor_itter : 1018
CB sor_itter : 633
Gause-Seidal : 290729


100-element Array{Float64,1}:
 103.4479486359771 
 103.63793147973331
 103.33740169154488
 102.14099518690315
 101.10989655952864
 100.14282190516575
 100.71549720893162
 100.86593907422267
 100.25218271357114
  99.78418087209002
 103.46400668907769
 103.08641999575767
 102.52163368394103
   ⋮               
  99.04640407718385
  98.66068709959447
 100.42586065774577
 100.52202698546581
 101.29702431918716
 101.60069541276138
 102.00449587101843
 101.88594889800058
 100.77135424876566
  99.50133608706352
  98.18605392298748
  97.7196383200333 